In [17]:
# !pip install -U dspy-ai litellm openai
import os, json, re, textwrap, datetime
import dspy, litellm

In [18]:
import importlib.metadata

print("dspy-ai:", importlib.metadata.version("dspy"))
print("litellm:", importlib.metadata.version("litellm"))
print("openai:", importlib.metadata.version("openai"))


dspy-ai: 3.0.3
litellm: 1.79.1
openai: 2.3.0


In [19]:
philocal = dspy.LM(
    model="phi4-local:latest",
    base_url="http://192.168.1.102:11434/v1",
    api_key="dummy",
    custom_llm_provider="openai",
    temperature=0.2
)



#dspy.LM(
#    "openai/llama-local",              # nom arbitraire côté client
#    api_base="http://localhost:8000/v1",
#    api_key="dummy",                   # requis par certaines libs, peut être vide
#    temperature=0.2
#)

In [20]:
philocal("Dites bonjour, répondez en un mot")

['Bonjour!']

### Clé API et connexion LLM

In [5]:
# Clé API
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")


In [6]:
api_key = os.getenv("OPENAI_API_KEY")
lm = dspy.LM(
#    model="gpt-5-mini-2025-08-07",          # modèle reconnu par l’API OpenAI
    model="gpt-5-mini",          # modèle reconnu par l’API OpenAI
    api_key=os.environ["OPENAI_API_KEY"],
    temperature=1.0,
    cache=True,
    max_tokens=16000        # <- obligatoire ici pour désigner le backend
)
#dspy.settings.cache = None
dspy.settings.configure(lm=lm)
lm("Dites OK, répondez en un mot")

['OK']

# DSPy

## Paramètres

In [7]:
themes = [
    "informatique quantique", 
    "Advaita Vedanta", 
    "politique de l'enfant unique", 
    "relativité et éther",
    "Théorème des accroissements finis",
    "Voyage au bout de la nuit",
    "DSPy",
    "Le mécanisme d'attention dans les transformers",
    "Des souris et des hommes",
    "La commission Warren",
    "Dix petits nègres",
    "Comment j'ai mangé mon grand-père",
    "La conquête de l'Ouest et le capitalisme",
    "La théorie des nombres en cryptographie",
    "Histoire de l'écriture",
    "Temps et physique quantique",
    "La conquête du pouvoir de Benito",
    "La marche de boudienny",
    "Le fusil M56T, manuel d'entretien",
        "phénoménologie de la perception",
    "la mécanique céleste de Laplace",
    "la conscience chez les animaux",
    "les camps soviétiques du Goulag",
    "le réalisme magique en littérature",
    "le paradoxe d'Olbers",
    "la musique dodécaphonique",
    "la dialectique hégélienne",
    "les utopies technologiques",
    "la physique des trous noirs",
    "la mémoire collective et l'identité",
    "l’éthique de Spinoza",
    "les réseaux de neurones convolutifs",
    "les grands incendies de Rome",
    "la notion de sacrifice chez les Aztèques",
    "le mythe d'Orphée",
    "les accords de Bretton Woods",
    "la stratégie militaire de Napoléon",
    "la cognition incarnée",
    "la classification des particules élémentaires",
    "l'art brut",
    "la vie de Nikola Tesla",
    "les effets de la dopamine",
    "la poétique d’Aristote",
    "les codes QR et la logistique moderne",
    "le roman épistolaire",
    "la révolution cognitive du Paléolithique",
    "l’histoire de l’alchimie",
    "la logique floue",
    "les illusions d'optique",
    "les expériences de Milgram",
    "la cybernétique de Norbert Wiener",
    "la symbolique des couleurs",
    "le procès de Socrate",
    "la lecture rapide et ses limites",
    "la biologie synthétique",
    "le concept de karma",
    "la cartographie ancienne",
    "le langage des abeilles",
    "les protocoles de consensus en blockchain",
    "le nihilisme russe au XIXe siècle",
    "la thermodynamique statistique",
    "le théâtre de l'absurde",
    "les mathématiques de Gödel",
    "la métaphysique bouddhiste",
    "le chant diphonique",
    "la transmission orale des savoirs",
    "l'intelligence des corvidés",
    "le cinéma expressionniste allemand",
    "le syndrome de l'imposteur"
]

In [8]:
categories = ["extrait", "commentaire", "synthese", "texte_libre"]

In [9]:
type_du_document_par_categorie = {
    "extrait": ["roman original", "article scientifique", "essai","roman","rapport", "article encyclopédique"],
    "commentaire": ["tweet", "article Wikipédia", "post de forum"],
    "synthese": ["rapport", "article encyclopédique", "ouvrage collectif","roman","manuel"],
    "texte_libre": ["note personnelle", "email", "discussion orale"],
    }


## Exemples

In [10]:
exemples = [
    {
        "categorie": "synthèse",
        "date": "2025-03-29 20:27:28",
        "type_du_document" : "rapport",
        "theme": "commission Warren",
        "expressions_clefs": ['assassinat', 'John F. Kennedy', 'Lee Harvey Oswald', 'conspiration', 'théories du complot', 'Earl Warren', 'rapport'],
        "contenu" : "La commission Warren a été créée en 1963 pour enquêter sur l'assassinat du président John F. Kennedy.\
                    Dirigée par le juge Earl Warren, elle a conclu que Lee Harvey Oswald était l'unique assassin\
                    et qu'il n'y avait pas de conspiration.\
                    Cependant, son rapport a été sujet à de nombreuses controverses et théories du complot,\
                    remettant en question la validité de ses conclusions.", 
        "url": "https://www.archives.gov/research/jfk/warren-commission-report"
    },
    {
        "categorie": "extrait",
        "date": "21/10/2023",
        "type_du_document":"manuel",
        "theme":"spiritualité",
        "expressions_clefs": ["Yi King"],
        "contenu": "Le feu rayonne Tong Ren est l’hexagramme de l’entente et de la relation\
                    à l’autre. Il nous invite à sortir de notre groupe d’amis ou de notre famille\
                    pour aller au-delà de nos frontières habituelles. C’est typiquement le moment\
                    de nous socialiser et de cultiver la convivialité. Nous devons nous créer un\
                    réseau de connaissances qui nous sorte de nos habitudes confortables. C’est\
                    également le moment d’apprendre à dire oui.\n",
        "url": "https://lire.amazon.fr/kp/kshare?asin=B00SN5MNP4&id=wbcez2aznzhaphbqwselvztfzm&ref_=r_sa_glf_b_0_hdrw_ss_AAu5AAA"
    },
    {
        "categorie": "commentaire",
        "date": "21/08/2023",
        "type_du_document":"tweet",
        "theme":"informatique",
        "expressions_clefs": ["raisonnement","espace latent"],
        "contenu": "Un tweet sur les espaces latents des raisonnements au sein des LLM",
        "url": "https://lire.amazon.fr/kp/kshare?asin=B00SN5MNP4&id=wbcez2aznzhaphbqwselvztfzm&ref_=r_sa_glf_b_0_hdrw_ss_AAu5AAA"
    },
    {
        "categorie": "synthèse",
        "date": "22/08/2023",
        "type_du_document":"article technique",
        "theme":"informatique",
        "expressions_clefs": ["ColBERT","architecture"],
        "contenu": "Un article clair sur Medium permettant de comprendre le fonctionnement et l'architecture de ColBERT d'Omar Khattab.\
                    L'article décrit, en particulier, les architectures concurrentes et pourquoi ColBERT est plus rapide.\
                    C'est un excellent article d'introduction à ColBERT.",
        "url": "https://pangyoalto.com/en/colbertv1-2-review-en/"
    },
    {
        "categorie": "texte_libre",
        "date": "21/08/2023",
        "type_du_document":"note personnelle",
        "theme":"informatique",
        "expressions_clefs": ["canal sécurisé","associations","petits modèles de langage"],
        "contenu": "Comment sauver sur une machine locale par un canal sécurisé (Session) des notes et\
        rechercher automatiquement des associations entre elles grâce à un petit modèle de langage (SLM).",
        "url": ""
    },
    {
        "categorie": "texte_libre",
        "date": "21/08/2023",
        "contenu": """
                    Le conte dit que, lorsqu’il eut fait justice de Méléagant, Lancelot demeura quelque temps à la cour, où il n’est joie et plaisir qu’il n’obtint de la reine sa mie. On lui apprit comment Bohor était arrivé et les prouesses qu’il avait faites contre cinq des meilleurs de la Table ronde ; puis comment le roi, la reine et tous les chevaliers étaient montés dans la charrette. Il en sourit de plaisir.
                    — Beau cousin, dit-il à Bohor, n’ayez pas si bien commencé pour laisser désormais toute chevalerie ; mais gardez, pour l’amour de moi, qu’aucune dame ou demoiselle requière jamais votre aide sans l’obtenir.
                    Personne n’avait osé lui annoncer la mort de Galehaut : aussi pensait-il que son ami était retourné dans sa terre.
                    — Dame, dit-il un jour à la reine, il me faut aller aux Iles lointaines : quand je saurai des nouvelles de mon compagnon, j’aurai le cœur plus joyeux.
                    Ah ! peu s’en fallut qu’elle ne lui avouât la vérité ! Mais elle songeait au chagrin qu’il aurait quand il apprendrait la mort de Galehaut, et son cœur se serra si fort qu’il s’en fallut de peu qu’elle ne pâmât.
                    — Beau doux ami, dit-elle seulement, que Notre Sire vous conduise !
                    Et, le lundi, Lancelot s’arma au sortir de la messe et partit si secrètement que personne ne le vit, hors sa dame qui avait monté sur la plus haute tour et le suivit longtemps des yeux.
                    Il chevaucha tant qu’il parvint dans une forêt haute, grande et agréable à y errer ; là, bien qu’il fît grand chaud, sous les arbres feuillus l’ombre était agréable. Il était fort en peine de son chemin, lorsqu’il découvrit des traces de chevaux : il les suivit à bonne allure et ne tarda pas à joindre une demoiselle qui chevauchait noblement sur un beau palefroi, suivie de ses gens ; elle était toute vieille et chenue, pourtant elle avait ses cheveux déliés sur les épaules à la façon des pucelles, et sur la tête une couronne de roses ; ainsi qu’il convient environ la Saint-Jean.
                    — Demoiselle, lui dit-il après l’avoir saluée, me sauriez-vous enseigner le chemin qui mène à la terre de Galehaut ?
                    — En nom Dieu, je puis bien vous l’enseigner, mais pourvu que vous m’accordiez un don : c’est que vous me suivrez dès que je vous en requerrai.
                    Lancelot octroya le don. Hélas ! il s’en repentit et chagrina assez quand le moment fut venu !
                    """,
            "type_du_document":"note personnelle",
            "theme":"roman de  chevalerie",
            "expressions_clefs": ["canal sécurisé","associations","petits modèles de langage"],
            "url": "https://fr.wikisource.org/wiki/Le_Ch%C3%A2teau_aventureux/1"
    }
]

In [11]:
len(exemples)

6

In [12]:
champs_a_supprimer = ["expressions_clefs"]
nouveaux_exemples = [
    {k: v for k, v in item.items() if k not in champs_a_supprimer}
    for item in exemples
]
print(json.dumps(nouveaux_exemples, ensure_ascii=False, indent=2, sort_keys=True))

[
  {
    "categorie": "synthèse",
    "contenu": "La commission Warren a été créée en 1963 pour enquêter sur l'assassinat du président John F. Kennedy.                    Dirigée par le juge Earl Warren, elle a conclu que Lee Harvey Oswald était l'unique assassin                    et qu'il n'y avait pas de conspiration.                    Cependant, son rapport a été sujet à de nombreuses controverses et théories du complot,                    remettant en question la validité de ses conclusions.",
    "date": "2025-03-29 20:27:28",
    "theme": "commission Warren",
    "type_du_document": "rapport",
    "url": "https://www.archives.gov/research/jfk/warren-commission-report"
  },
  {
    "categorie": "extrait",
    "contenu": "Le feu rayonne Tong Ren est l’hexagramme de l’entente et de la relation                    à l’autre. Il nous invite à sortir de notre groupe d’amis ou de notre famille                    pour aller au-delà de nos frontières habituelles. C’est typiquement le mo

## Générateur DSPy de parangons

In [13]:
class GenerateNote(dspy.Signature):
    """Génération, en français, d'une note à partir de trois entrées principales : un thème, une catégorie, et le type de sa source. 
    
- categorie : Une note peut être un commentaire sur tout ou partie d'un document (un livre, un article, un courrier, une page web, un film,
une oeuvre musicale, etc.), un extrait ou une synthèse d'un tel document, ou bien un texte libre. 
- theme : Quand la catégorie de la note est un texte libre son thème est ce dont il traite dans le cas contraire le thème est ce dont traite le
  document. Le thème se présente comme une courte chaîne de caractères, par exemple : "La théorie des nombres en cryptographie", 
  "Histoire de l'écriture", "Temps et physique quantique", "La conquête du pouvoir de Benito", "La marche de boudienny",
  "Le fusil M56T, manuel d'entretien", "phénoménologie de la perception", "la mécanique céleste de Laplace", "la conscience chez les animaux",
- type du document: il précise si le document est un roman, un courrier (email ou autre), un film, etc. Pour un texte libre ce champ est nul.
- expressions_clés : ce sont des mots ou des expressions apparaissant dans le texte du contenu, une expression au minimum est nécessaire
  elles doivent apparaître verbatim dans le champ contenu. 
- contenu : C'est le contenu proprement dit de la note qui doit être généré. Ce contenu ne doit pas faire directement référence ni à 
  la catégorie ni au thème. Par exemple, si la catégorie est "commentaire", et le thème "La guerre des Gaules" le contenu ne doit pas 
  débuter pas par : "Commentaire sur la guerre des Gaules". En résumé, si le contenu reflète thème et la catégorie, ils ne les contient pas
  explicitement. Cette règle est importante car les notes générées pourront être utilisées à classifier des textes par thèmes ou catégories.

Éléments de forme et règles additionnelles
- Langue : français.

=== CONTRAT IMMUTABLE — NE PAS MODIFIER ===
Rôles des champs :
- theme : ENTRÉE
- categorie : ENTRÉE
- type_du_document : ENTRÉE (null si texte_libre)
- contenu : SORTIE (ne doit pas faire directement référence ni à la catégorie ni au thème).
- url : SORTIE (elle doit être inventée, elle doit respecter la syntaxe d'une url)
- date : SORTIE (YYYY-MM-DD) (elle doit être inventée)
- expressions_clefs : SORTIE UNIQUEMENT (jamais demandé en entrée elles ne sont pas directement générées mais extraites du champ contenu).
Toute version qui contredit ce contrat est nulle et doit être rejetée.
=== FIN CONTRAT IMMUTABLE ===

CONTRAINTE DSPy :
- Sortir uniquement : contenu (string), url (string|null), date (YYYY-MM-DD), expressions_clefs (array de strings), type_de_document, theme, categorie.
- `expressions_clefs` doit apparaître à l’identique dans `contenu`.
- Ne pas solliciter d’entrées supplémentaires (jamais demander des expressions clés).
- L'url et la date doivent être inventées.


But final
Générer une note exploitable pour l'affinage (fine tuning) d'un LLM dans le bus de classifier des notes de lecture ou en extraire de expressions clés caractéristiques.
"""
    
    theme:str = dspy.InputField(desc="Thème principal de la note")
    categorie:str = dspy.InputField(desc="Type de note: 'extrait', 'commentaire', 'synthese', ou 'texte_libre' ")
    type_du_document: str = dspy.InputField(desc="Type du document source: article Wikipédia, roman original, tweet, billet de blog, etc.")
    
    contenu:str = dspy.OutputField(desc="Contenu de la note, ce contenu doit être en accord avec le thème 'theme' mais bien sûr aussi avec la catégorie 'categorie' et le type de document source_type")
    url:str = dspy.OutputField(desc="l'url ou la référence du document au sujet duquel la note est écrite")
    date:str = dspy.OutputField(desc="Date d'écriture de la note est une date au format ISO totalement imaginaire")
    expressions_clefs:list = dspy.OutputField(desc="Liste d'expressions catactéristiques apparaissant dans le champ contenu")
    
class NoteGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate = dspy.Predict(GenerateNote)
    
    def forward(self, categorie, theme, type_du_document):
        result = self.generate(categorie=categorie, theme=theme, type_du_document=type_du_document)
        print(f"{categorie}")
        return {
            "contenu": result.contenu,
            "expressions_clés": result.expressions_clefs,
            "url": result.url,
            "date": result.date,
            "categorie": categorie,
            "theme": theme,
            "type_du_document": type_du_document
        }



## Comparateur de textes pour la métrique sémantique
Pour deux textes (text_a, text_b) produit UNE note globale 1..5
(thème + ton + style). 5 = très ressemblant, 1 = très différent.


In [14]:
# ---
# DSPy : comparaison 2 textes → score global 1..5, avec LM particulier
# Correction : ne jamais passer lm= à dspy.Predict ; utiliser un context manager.
# ---

import dspy

# 1) Définir un LM particulier (ok dspy.LM)
lm_comparateur = dspy.LM(
    model="gpt-4o-mini",   # ajustez le nom du modèle
    temperature=0.0,
    cache=True,
    max_tokens=128
)

# 2) Signature -> un seul entier 1..5
class CompareTextsGlobal(dspy.Signature):
    """
    Compare deux textes (text_a, text_b) et produit UNE note globale 1..5
    (thème + ton + style). 5 = très ressemblant, 1 = très différent.
    Sortie : un entier 1..5, sans commentaire.
    """
    text_a: str = dspy.InputField(desc="Premier texte")
    text_b: str = dspy.InputField(desc="Second texte")
    score: int  = dspy.OutputField(desc="Entier 1..5, score global")

# 3) Module : pas de lm passé à Predict ; on l'applique via un contexte au moment de l'appel
class TextComparatorGlobal(dspy.Module):
    def __init__(self, lm: dspy.LM | None = None):
        super().__init__()
        self.pred = dspy.Predict(CompareTextsGlobal)  # <- pas de lm ici
        self.lm = lm

    def forward(self, text_a: str, text_b: str) -> int:
        # On utilise le LM voulu via un context manager (pas sérialisé dans la requête)
        if self.lm is not None:
            with dspy.settings.context(lm=self.lm):
                out = self.pred(text_a=text_a, text_b=text_b)
        else:
            # Sinon, ça utilisera le LM global configuré ailleurs
            out = self.pred(text_a=text_a, text_b=text_b)

        # garde-fou : cast/clamp 1..5
        try:
            s = int(getattr(out, "score", 3))
        except Exception:
            s = 3
        return max(1, min(5, s))

# 4) Exemple d’utilisation avec LM local
comparator = TextComparatorGlobal(lm=lm_comparateur)

text1 = "Article expliquant la politique de l'enfant unique en Chine."
text2 = "Synthèse historique de la politique démographique chinoise au XXe siècle."

score = comparator.forward(text1, text2)
print("Score global (1..5):", score)


2025/11/18 16:29:14 WARNING dspy.primitives.module: Calling module.forward(...) on TextComparatorGlobal directly is discouraged. Please use module(...) instead.


Score global (1..5): 3


In [15]:
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union
Json = Union[str, Dict[str, Any]]

compteur_global = 0;
def mentions_expr_as_input(text: str) -> bool:
    return bool(re.search(
        r"(fournir|donnez|veuillez|merci de).*(expression[_ -]?cl[ée]s?)",
        text, flags=re.I
    ))


def semantic_metric(
    gold: dict,
    pred: dict,
    trace: dict | None = None,
    pred_name: str | None = None,
    pred_trace: dict | None = None,
    *,
    w_semantic: float = 1.0,   # mettez 1.0 si vous n'utilisez pas l'expression
    w_expr: float = 0.0,
) -> float:
    """Score [0,1] basé sur le comparateur DSPy (1..5 → /5)."""
    gold_text = str((gold or {}).get("contenu") or "")
    pred_text = str((pred or {}).get("contenu") or "")

    if not gold_text or not pred_text:
        return 0.1  # neutre faible si vide

    if mentions_expr_as_input(pred_text):
        return 0.0  # rejet net
    try:
        # Utilisez l’opérateur d’appel, pas .forward (évite l’avertissement)
        score_15 = comparator(text_a=gold_text, text_b=pred_text)
        s_semantic = score_15 / 5.0
    except Exception:
        print(f"Erreur de comparaison entre {gold_text} et {pred_text}") 
        s_semantic = 0.6  # neutre moyen si échec LLM

    score = float(s_semantic)  # ici w_expr=0.0, donc score = s_semantic

    if isinstance(trace, dict):
        trace.update({
            "semantic_raw_1_5": score_15 if 'score_15' in locals() else None,
            "semantic": s_semantic,
            "final": score,
            "pred_name": pred_name,
        })
    if isinstance(pred_trace, dict):
        pred_trace.update({"final": score})

    global compteur_global
    compteur_global +=  1
    if compteur_global % 10 == 0:
        print(f"******************************************** au tour {compteur_global} score = {score}")
    return score


Notre but est d'abord d'obtenir un prompt amélioré par GEPA.
L'utilisation de gpt-4o-mini est cachée donc l'entrainement est très rapide.


In [ ]:
from dspy.teleprompt import GEPA

trainset = [
    dspy.Example(
        categorie=ex["categorie"],
        theme=ex["theme"],
        type_du_document=ex["type_du_document"],
        contenu=ex["contenu"],
        url=ex["url"],
        date=ex["date"],
        expressions_clefs=ex["expressions_clefs"],
    ).with_inputs("categorie", "theme", "type_du_document")
    for ex in exemples
]
print(len(trainset))

#for i, ex in enumerate(trainset):
#    try:
#        out = generator.forward(**ex.inputs())
#        print(f"✅ Exemple {i} OK")
#    except Exception as e:
#        print(f"❌ Erreur à l'exemple {i} : {e}")


# Initialisation et entraînement
generator = NoteGenerator()

teleprompter = dspy.GEPA(
    metric=semantic_metric,
    reflection_lm=lm,
    # Option A : profil auto
    #auto="heavy",

    # Option B : borne explicite (choisir l'une des deux lignes suivantes)
    #max_full_evals=1,          # très court (démonstration)
    max_metric_calls=1000,    # borne sur le nb total d'appels métrique
    track_stats=True,
    track_best_outputs=True,
)
"""teleprompter = dspy.GEPA(
    metric=semantic_metric,
    reflection_lm=lm,
    auto="heavy",
    iterations=200,         # nombre d’itérations max
    patience=10,            # stop après 10 itérations sans progrès
    max_metric_calls=500,   # borne dure sur nb appels métrique
    track_stats=True,
    track_best_outputs=True
)"""

try:
    compiled_generator = teleprompter.compile(generator, trainset=trainset)
except RateLimitError as e:
    print("Quota dépassé, réessayez plus tard :", e)


2025/11/18 16:29:28 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 1000 metric calls of the program. This amounts to 166.67 full evals on the train set.
2025/11/18 16:29:28 INFO dspy.teleprompt.gepa.gepa: Using 6 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.


6


GEPA Optimization:   0%|          | 0/1000 [00:00<?, ?rollouts/s]

extrait
texte_libre
commentaire
texte_libre
synthèse
synthèse


2025/11/18 16:30:14 INFO dspy.evaluate.evaluate: Average Metric: 2.8000000000000003 / 6 (46.7%)
2025/11/18 16:30:14 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.46666666666666673
GEPA Optimization:   1%|          | 6/1000 [00:45<2:04:52,  7.54s/rollouts]2025/11/18 16:30:14 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.46666666666666673


...

2025/11/18 17:29:37 INFO dspy.teleprompt.gepa.gepa: Iteration 51: Proposed new text for generate: Vous êtes un assistant chargé de générer, en français, une "note" utilisable pour l'affinage (fine‑tuning) d'un modèle de langue destiné à classer des notes de lecture et à en extraire des expressions‑clefs. Remplacez et explicitez toute consigne antérieure : suivez ces règles strictes sans exception. Toute sortie non conforme doit être remplacée par exactement la chaîne ERREUR_INTERNE (seule et sans espaces ni sauts de ligne).

1) Format d'entrée (fourni par l'appelant)
   - categorie (string) — ex. "texte_libre", "commentaire", "extrait", "synthese", etc.
   - theme (string) — courte chaîne descriptive ; conserver exactement majuscules, accents, espaces.
   - type_du_document (string|null) — ex. "roman", "manuel", "courrier", "film", "note personnelle", "tweet"; peut valoir null.
   - Remarque : ces trois valeurs sont fournies par l'appelant et doivent être reproduites exactement en sort

texte_libre
commentaire
synthèse


2025/11/18 17:30:17 INFO dspy.evaluate.evaluate: Average Metric: 1.4000000000000001 / 3 (46.7%)
2025/11/18 17:30:17 INFO dspy.teleprompt.gepa.gepa: Iteration 51: New subsample score is not better, skipping
GEPA Optimization:  34%|███▎      | 336/1000 [1:00:48<1:54:02, 10.30s/rollouts]2025/11/18 17:30:17 INFO dspy.teleprompt.gepa.gepa: Iteration 52: Selected program 4 score: 0.5333333333333333


  0%|          | 0/3 [00:00<?, ?it/s]synthèse
texte_libre
******************************************** au tour 490 score = 0.8
Average Metric: 1.60 / 3 (53.3%): 100%|██████████| 3/3 [00:00<00:00, 43.14it/s]

2025/11/18 17:30:17 INFO dspy.evaluate.evaluate: Average Metric: 1.6 / 3 (53.3%)


In [ ]:
compiled_generator

In [ ]:
res = compiled_generator(categorie="soulignement", theme="spiritualité",type_du_document="manuel d'apprenstissage")
print(res)
res = compiled_generator(categorie="commentaire", theme="physique", type_du_document="article arxicv")
print(res)
res = compiled_generator(categorie="brouillon", theme="écriture d'un passage de roman",type_du_document=None)
print(res)
res = compiled_generator(categorie="synthese", theme="Jeanne d'Arc",type_du_document="essai historique")
print(res)
res = compiled_generator(categorie="soulignement", theme="X un langage pour l'informatique quantique",type_du_document="manuel d'utilisation")
print(res)
res = compiled_generator(categorie="texte_libre", theme="Comment j'ai mangé mon grand-père",type_du_document="roman")
print(res)

In [ ]:
prompt_gepa = """ But = Générer, en français, une note exploitable pour l'affinage (fine‑tuning) d'un LLM chargé de classifier des notes de lecture et/ou d'en extraire des expressions‑clés caractéristiques.

1) Format d'entrée (NE PAS MODIFIER)
- Entrées fournies (ne doivent pas être altérées) :
  - theme (string) : donne le sujet du document. Pour categorie = "texte_libre", theme indique de quoi parle le texte libre ; sinon theme indique de quoi traite le document.
  - categorie (string) : donne la nature de la note (ex. "commentaire", "synthèse", "extrait", "brouillon", "texte_libre", "note personnelle", "soulignement", etc.).
  - type_du_document (string|null) : précise le type du document source ("roman", "article technique", "film", "manuel", "courrier", etc.). Si categorie == "texte_libre", cette valeur peut être null.

2) CONTRAT IMMUTABLE — règles absolues
- Rôles des champs : theme, categorie, type_du_document sont ENTRÉES ; contenu, url, date, expressions_clefs sont SORTIES.
- Ne jamais changer la nature (entrée/sortie) ni les noms des champs.
- Langue de sortie : français exclusivement.
- Ne demandez jamais d'informations supplémentaires à l'utilisateur.
- Ne générez aucun texte hors de l'objet JSON de sortie (voir section 5).

3) Format strict de sortie (JSON unique)
- La réponse finale doit contenir uniquement UN OBJET JSON (aucun texte avant/après).
- L'objet JSON doit contenir exactement ces clés (ni plus, ni moins), avec ces noms précis :
  - contenu (string)
  - url (string|null)
  - date (string) — format YYYY-MM-DD
  - expressions_clefs (array of strings)
  - type_de_document (string|null) — copier strictement la valeur d'entrée `type_du_document`
  - theme (string) — recopier la valeur d'entrée `theme`
  - categorie (string) — recopier la valeur d'entrée `categorie`
- Respecter les types JSON corrects : null (non string) pour valeurs nulles, tableau JSON pour expressions_clefs.

4) Règles détaillées pour les champs de sortie
- contenu :
  - Rédiger en français, naturel, fluide.
  - Adapter le style et la longueur à la valeur de `categorie` :
    - synthèse : commencer idéalement par "Synthèse de [type_de_document] sur [theme] :" (si type_de_document est null, omettre ou écrire "Synthèse sur [theme] :"). 3–6 phrases, claires et structurées, couvrant idées principales, points clés et conclusion/recommandation.
    - commentaire : recommander d'ouvrir par "Commentaire — [theme] :" suivi de 3–8 phrases analytiques et critiques, indiquant forces/faiblesses et exemples concrets.
    - extrait : fournir un passage de 1–4 phrases qui peut se présenter comme extrait du document (les guillemets sont permis mais restent dans la même chaîne). Au moins une des expressions dans expressions_clefs doit être une phrase courte tirée textuellement du contenu.
    - brouillon / note personnelle : style plus informel, possible usage de listes à puces ou tirets, phrases courtes ; cependant tout doit rester dans une seule chaîne JSON.
    - texte_libre : texte autonome, cohérent avec `theme`, 3–8 phrases, narratif ou analytique.
    - soulignement / autres catégories similaires : adapter la longueur à 2–6 phrases mettant en avant points saillants.
  - Inclure explicitement (au moins une fois) chacune des expressions qui figureront dans expressions_clefs (les occurrences doivent être verbatim).
  - Varier le vocabulaire et insérer quelques expressions caractéristiques du thème pour améliorer la diversité d'entraînement.
  - Éviter tout formatage lourd (pas de Markdown, pas d'en‑têtes HTML). Contenu = chaîne texte simple.

- expressions_clefs :
  - Extraire entre 1 et 8 expressions/phrases courtes (préférence 2–6).
  - Chaque élément doit apparaître textuellement (au mot près) dans `contenu`.
  - Choisir des éléments distinctifs, représentatifs du thème, points clés ou termes techniques.
  - Les expressions peuvent être des mots simples ou groupes de mots, ou courtes phrases (pour "extrait" au moins une phrase).
  - Ne jamais demander ces expressions à l'utilisateur ; elles doivent provenir du texte généré.

- url :
  - Inventer une URL plausible et syntaxiquement valide (schéma http:// ou https://). Ex. "https://domain.example/path".
  - Pour categorie = "texte_libre" ou quand l'absence est pertinente, url peut être null. Si null, utiliser la valeur JSON null (pas "None" ni "null" en string).

- date :
  - Inventer une date plausible au format YYYY-MM-DD.
  - Varier les années raisonnablement (préférence pour 2018–2030).
  - Respecter le format exact (ex. "2026-04-17").

- type_de_document :
  - Copier exactement la valeur d'entrée `type_du_document` (même chaîne). Si l'entrée est null, mettre null.

- theme et categorie :
  - Copier exactement les valeurs d'entrée (même chaîne).

5) Contraintes de production
- La sortie doit être UNique : uniquement l'objet JSON (aucune ligne explicative, pas de code block, pas d'annotations).
- Vérifications obligatoires à faire avant sortie :
  - L'objet JSON contient exactement les clés demandées ; aucune clé supplémentaire.
  - Chaque chaîne de `expressions_clefs` apparaît verbatim dans `contenu`.
  - `type_de_document` correspond exactement à `type_du_document` d'entrée (ou null).
  - Langue = français ; `date` au format YYYY-MM-DD ; `url` syntaxe valide si non null.
  - Longueur et ton du `contenu` conformes à la `categorie`.

6) Erreurs à ne jamais commettre (exemples d'erreurs observées)
- Ne pas renvoyer "None" ou "None" en string pour url ; utiliser null JSON.
- Ne pas omettre des expressions_clefs dans le contenu ; elles doivent y figurer textuellement.
- Ne pas modifier les noms de champs d'entrée/sortie.
- Ne pas produire du texte hors de l'objet JSON.

7) Exemples de validation finale (checklist avant renvoi)
- Clés JSON exactes et types corrects.
- Chaque expression clé visible textuellement dans contenu.
- type_de_document identique à type_du_document d'entrée.
- Date au format YYYY-MM-DD, URL valide ou null.
- Langue française, longueur/contenu conformes à la catégorie.

Respecte strictement ce cahier des charges pour chaque requête. Ne produis aucune phrase explicative en dehors de l'objet JSON demandé.
"""

In [ ]:
class SignatureEntreeCorpus(dspy.Signature):
    f"""Utiliser le prompt ci-dessous pour produire strictement un objet JSON (7 clés).
    {prompt_gepa}
    Le modèle doit suivre ces instructions telles quelles, sans ajouter de texte hors JSON.
    """

    theme:str = dspy.InputField(desc="Thème principal de la note")
    categorie:str = dspy.InputField(desc="Type de note: 'extrait', 'commentaire', 'synthese', ou 'texte_libre' ")
    type_du_document: str = dspy.InputField(desc="Type du document source: article Wikipédia, roman original, tweet, billet de blog, etc.")
    
    contenu:str = dspy.OutputField(desc="Contenu de la note, ce contenu doit être en accord avec le thème 'theme' mais bien sûr aussi avec la catégorie 'categorie' et le type de document source_type")
    url:str = dspy.OutputField(desc="l'url ou la référence du document au sujet duquel la note est écrite")
    date:str = dspy.OutputField(desc="Date d'écriture de la note est une date au format ISO totalement imaginaire")
    expressions_clefs:list = dspy.OutputField(desc="Liste d'expressions catactéristiques apparaissant dans le champ contenu")

class GenerationEntreeCorpus(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate = dspy.Predict(SignatureEntreeCorpus) 

    def forward(self, theme: str, categorie: str, type_du_document: str):
        result = self.generate(categorie=categorie, theme=theme, type_du_document=type_du_document)
        #print(f"{categorie}")
        #print(f"{result}")
        return {
            "contenu": result.contenu,
            "url": result.url,
            "date": result.date,
            "expressions_clés": result.expressions_clefs,
            "categorie": categorie,
            "theme": theme,
            "type_du_document": type_du_document
        }
        

    


In [ ]:
gen = GenerationEntreeCorpus()

In [ ]:
gen(categorie="extrait", theme="spiritualité",type_du_document="manuel d'apprentissage")

In [ ]:
import time
import random

generated_notes = []
output_path = "corpus-10.txt"
nbnotes = 10
nbbatch = 5
stime = time.time()
with open(output_path, "w", encoding="utf-8") as f:
    for i in range(nbnotes):
        cat = random.choice(categories)
        st = random.choice(type_du_document_par_categorie[cat])
        th = random.choice(themes)
        if i % nbbatch == 0 and i != 0:
            
            print(f"{i} / {nbnotes}  {(time.time()-stime)}")
            for gnote in generated_notes:
                gnote = f"{gnote},\n"
                print(f"{gnote}")
                f.write(gnote)
            generated_notes = []
            
        note = gen(categorie=cat, theme=th,type_du_document=st)
        note["date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        note["categorie"] = cat
        note["source_type"] = st
        note["theme"] = th
        generated_notes.append(note)
        
print(f"Corpus sauvegardé en {(time.time()-stime)}s dans {output_path}")

## Ancienrun



In [ ]:
from dspy.teleprompt import GEPA

trainset = [
    dspy.Example(
        categorie=ex["categorie"],
        theme=ex["theme"],
        type_du_document=ex["type_du_document"],
        contenu=ex["contenu"],
        url=ex["url"],
        date=ex["date"],
    ).with_inputs("categorie", "theme", "type_du_document")
    for ex in exemples
]
print(len(trainset))

#for i, ex in enumerate(trainset):
#    try:
#        out = generator.forward(**ex.inputs())
#        print(f"✅ Exemple {i} OK")
#    except Exception as e:
#        print(f"❌ Erreur à l'exemple {i} : {e}")


# Initialisation et entraînement
generator = NoteGenerator()

teleprompter = dspy.GEPA(
    metric=semantic_metric,
    reflection_lm=lm,
    # Option A : profil auto
    auto="heavy",

    # Option B : borne explicite (choisir l'une des deux lignes suivantes)
    #max_full_evals=1,          # très court (démonstration)
    # max_metric_calls=500,    # borne sur le nb total d'appels métrique

    track_stats=True,
    track_best_outputs=True,
)

dspy.settings.cache = None
try:
    compiled_generator = teleprompter.compile(generator, trainset=trainset)
except RateLimitError as e:
    print("Quota dépassé, réessayez plus tard :", e)